In [ ]:
import numpy as np
import pandas as pd
import cv2
import os

path = 'C:/Users/Prateek Jain/Desktop/12/English/Hnd/Img/'
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 
          ]
folders = os.listdir(path)
folders.sort()

data = np.zeros((3410, 4096), dtype = float)
target = np.zeros((3410), dtype = str)
i = 0
j = 0
for fold in folders:
    curr_path = path + fold +'/'
    files = os.listdir(curr_path)
    for file in files:
        im = cv2.imread(curr_path+file, 0)
        im1 = cv2.resize(im, (64, 64))
        data[i] = im1.reshape((1,4096))
        i = i+1

j = 0
for i in range(0,3410,55):
    target[i:i+55] = labels[j]
    j = j + 1

In [ ]:
image = data[550:,:]
target = target[550:]

In [ ]:
from sklearn.cross_validation import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(image, target, test_size = 0.2, random_state = 42)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4, 4, figsize = (6, 6))
fig.subplots_adjust(hspace = 0.3)
for counter, ax in enumerate(axes.flat):
    ax.imshow(xtrain[counter].reshape((64,64)) , interpolation = 'nearest', cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("True {}".format(ytrain[counter]))

plt.show()

In [9]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Sequential

In [ ]:
input_shape = (64, 64, 1)
no_of_classes = 62

In [ ]:
xtrain.astype('float32')
xtest.astype('float32')

xtrain = xtrain / 255
xtest = xtest / 255

y_train_one_hot_encoded = keras.utils.to_categorical(ytrain , no_of_classes)
y_test_one_hot_encoded = keras.utils.to_categorical(ytest , no_of_classes)

In [ ]:
xtrain = xtrain.reshape((xtrain.shape[0], 64, 64, 1))
xtest = xtest.reshape((xtest.shape[0], 64, 64, 1))

xtrain.shape

In [2]:
model = Sequential()

In [7]:
model.add(Conv2D(64, kernel_size=(3,3) , input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(no_of_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 64)        640       
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        36928     
__________

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(xtrain, y_train_one_hot_encoded,
          batch_size=128,
          epochs=15,
          validation_data=(xtest, y_test_one_hot_encoded))

In [ ]:
score = model.evaluate(xtest, y_test_one_hot_encoded, verbose=0)
score

In [ ]:
model.save('model-chars.h5')